In [187]:
#packages
import yfinance as yf
import pandas as pd

In [196]:
'''
start and end date format'%Y-%m-%d'
valid periods ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
'''
class Yahoo_scraper:
    def __init__(self, ticker_name: str = 'EURINR=X', start_date: str = None, end_date: str = None, interval: str ='1d'):
        self.ticker_name = ticker_name
        self.start_date =start_date
        self.end_date = end_date
        self.interval = interval
    def historical_data(self)->pd.DataFrame:
        stock = yf.Ticker(self.ticker_name)
        data = stock.history(start = self.start_date,end = self.end_date,interval = self.interval)
        required_data = data.loc[:,['Open','High','Low','Close']]
        return required_data
        

In [195]:
# test_object = Yahoo_scraper(start_date = '2024-10-20',end_date = None,interval = '1d')
# test_object.historical_data()